<h1 align=center>Capítulo 5</h1>
<h2 align=center>Arquivos e I/O</h2>
<p align= center><img src=https://t.ctcdn.com.br/RhgceU3jDPpm_52ctxpo1RRZ3AQ=/0x36:799x486/512x288/smart/filters:format(webp)/i333320.jpeg width=500</p>

Todos os programas precisam executar entrada e saída. Este capítulo abrange idiomas comuns para trabalhar com diferentes tipos de arquivos, incluindo arquivos de texto e binários, codificações de arquivos e outros assuntos relacionados. Técnicas para manipular nomes de arquivos e diretórios também são cobertas.

## 5.1. Lendo e escrevendo dados de texto

**Problema**

Você precisa ler ou gravar dados de texto, possivelmente em diferentes codificações de texto, como ASCII, UTF-8 ou UTF-16.

**Solução**

Use a função `open()` com o modo **rt** para ler um arquivo de texto. Por exemplo:

In [2]:
# Read the entire file as a single string
with open('somefile.txt', 'rt', encoding='utf8') as f:
    data = f.read()
    print(data)

As sem-razões do amor

Eu te amo porque te amo,
Não precisas ser amante,
e nem sempre sabes sê-lo.
Eu te amo porque te amo.
Amor é estado de graça
e com amor não se paga.

Amor é dado de graça,
é semeado no vento,
na cachoeira, no eclipse.
Amor foge a dicionários
e a regulamentos vários.

Eu te amo porque não amo
bastante ou demais a mim.
Porque amor não se troca,
não se conjuga nem se ama.
Porque amor é amor a nada,
feliz e forte em si mesmo.

Amor é primo da morte,
e da morte vencedor,
por mais que o matem (e matam)
a cada instante de amor.


In [5]:
# Iterate over the lines of the file
with open('somefile.txt', 'rt', encoding='utf8') as f:
    for line in f:
        # process line
        print(line)


As sem-razões do amor



Eu te amo porque te amo,

Não precisas ser amante,

e nem sempre sabes sê-lo.

Eu te amo porque te amo.

Amor é estado de graça

e com amor não se paga.



Amor é dado de graça,

é semeado no vento,

na cachoeira, no eclipse.

Amor foge a dicionários

e a regulamentos vários.



Eu te amo porque não amo

bastante ou demais a mim.

Porque amor não se troca,

não se conjuga nem se ama.

Porque amor é amor a nada,

feliz e forte em si mesmo.



Amor é primo da morte,

e da morte vencedor,

por mais que o matem (e matam)

a cada instante de amor.


Da mesma forma, para escrever um arquivo de texto, use `open()` com o modo **wt** para escrever um arquivo, limpando e sobrescrevendo o conteúdo anterior (se houver). Por exemplo:

In [7]:
# Write chunks of text data
text1 = 'Esse é um texto para teste.'
text2 = 'Esse é um outro texto para teste'
with open('otherfile.txt', 'wt') as f:
    f.write(text1)
    f.write(text2)

~~~python
# Redirected print statement
with open('otherfile.txt', 'wt') as f:
    print(line1, file=f)
    print(line2, file=f)
    ...
~~~

Para anexar ao final de um arquivo existente, use `open()` com mode **at**. Por padrão, os arquivos são lidos/gravados usando a codificação de texto padrão do sistema, como pode ser encontrado em `sys.getdefaultencoding()`. Na maioria das máquinas, isso é definido como utf-8. Se você sabe que o texto que está lendo ou escrevendo está em uma codificação diferente, forneça o parâmetro de codificação opcional para `open()`. Por exemplo:

~~~python
with open('somefile.txt', 'rt', encoding='latin-1') as f:
    ...
~~~

In [9]:
import sys
sys.getdefaultencoding()

'utf-8'

Python entende várias centenas de codificações de texto possíveis. No entanto, algumas das codificações mais comuns são **ascii, latin-1, utf-8 e utf-16**. O **UTF-8** geralmente é uma aposta segura se estiver trabalhando com aplicativos da web. **ascii** corresponde aos caracteres de 7 bits no intervalo U+0000 a U+007F. **latin-1** é um mapeamento direto de bytes 0-255 para caracteres Unicode U+0000 para U+00FF. A codificação **latin-1** é notável por nunca produzir um erro de decodificação ao ler o texto de uma codificação possivelmente desconhecida. Ler um arquivo como **latin-1** pode não produzir uma decodificação de texto completamente correta, mas ainda pode ser suficiente para extrair dados úteis dele. Além disso, se você gravar os dados posteriormente, os dados de entrada originais serão preservados.

**Discussão**

Ler e escrever arquivos de texto normalmente é muito simples. No entanto, há uma série de aspectos sutis para manter em mente. Primeiro, o uso da instrução `with` nos exemplos estabelece um contexto no qual o arquivo será usado. Quando o controle sai do bloco com, o arquivo será fechado automaticamente. Você não precisa usar a instrução `with`, mas se não usá-la, lembre-se de fechar o arquivo:

In [10]:
f = open('somefile.txt', 'rt')
data = f.read()
f.close()

Outra complicação menor diz respeito ao reconhecimento de novas linhas, que são diferentes no Unix e no Windows (ou seja, **\n** versus **\r\n**). Por padrão, o Python opera no que é conhecido como modo “nova linha universal”. Nesse modo, todas as convenções comuns de nova linha são reconhecidas e os caracteres de nova linha são convertidos em um único caractere **\n** durante a leitura.

Da mesma forma, o caractere de nova linha **\n** é convertido no caractere de nova linha padrão do sistema na saída. Se você não quiser essa tradução, forneça o argumento `newline=''` para `open()`, assim:
~~~python
# Read with disabled newline translation
with open('somefile.txt', 'rt', newline='') as f:
    ...
~~~

Para ilustrar a diferença, aqui está o que você verá em uma máquina Unix se ler o conteúdo de um arquivo de texto codificado pelo Windows contendo os dados brutos `hello world!\r\n`:

In [17]:
# Newline translation enabled (the default)
f = open('hello.txt', 'rt', encoding='utf8')
f.read()

'Olá Mundo!!\n'

In [18]:
# Newline translation disabled
g = open('hello.txt', 'rt', newline='', encoding='utf8')
g.read()

'Olá Mundo!!\r\n'

Uma questão final diz respeito a possíveis erros de codificação em arquivos de texto. Ao ler ou gravar um arquivo de texto, você pode encontrar um erro de codificação ou decodificação. Por exemplo:
~~~python
f = open('sample.txt', 'rt', encoding='ascii')
f.read()
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "/usr/local/lib/python3.3/encodings/ascii.py", line 26, in decode
        return codecs.ascii_decode(input, self.errors)[0]
 UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position
12: ordinal not in range(128)
>>>
~~~

Se você receber esse erro, geralmente significa que você não está lendo o arquivo na codificação correta. Você deve ler cuidadosamente a especificação do que quer que esteja lendo e verificar se está fazendo certo (por exemplo, lendo dados como UTF-8 em vez de Latin-1 ou o que for necessário). Se erros de codificação ainda são uma possibilidade, você pode fornecer um argumento opcional errors para open() para lidar com os erros. Aqui estão alguns exemplos de esquemas comuns de tratamento de erros:

In [20]:
>>> # Replace bad chars with Unicode U+fffd replacement char
>>> f = open('sample.txt', 'rt', encoding='ascii', errors='replace')
>>> f.read()

'Spicy Jalape��o'

In [21]:
>>> # Ignore bad chars entirely
>>> g = open('sample.txt', 'rt', encoding='ascii', errors='ignore')
>>> g.read()

'Spicy Jalapeo'

Se você está constantemente mexendo nos argumentos de codificação e erros para `open()` e fazendo muitos hacks, provavelmente está tornando a vida mais difícil do que precisa. A regra número um com o texto é que você simplesmente precisa ter certeza de que está sempre usando a codificação de texto adequada. Em caso de dúvida, use a configuração padrão (normalmente UTF-8).